In [1]:
import gensim
import os
from tensorflow import keras

In [2]:
import mysql.connector

In [3]:
import re

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [15]:
# load question type classifier
q_type_classifier = keras.models.load_model('model_full.h5')

In [6]:
# load doc2vec embeddings to embed input questions
doc2vec_embeddings = gensim.models.Doc2Vec.load('doc_model_full.h5')

In [7]:
# load knowledge graph database cursor
cnx = mysql.connector.connect(user='hadoop', password='lK3pA!2j', host="127.0.0.1", database="ner")
cursor = cnx.cursor()

In [8]:
sql_cmd = """SELECT dsynID, name FROM dsynNames;"""
cursor.execute(sql_cmd)

dsynNames = []
for item in cursor:
    dsynNames.append(item)

In [9]:
df = pd.DataFrame(dsynNames, columns=['dsynID', 'label'])

In [47]:
df[15:25]

,dsynID,label
15,2,mob ua npaws
16,2,typhoid fever
17,2,enteric fever
18,2,tus mob paratyphoid
19,2,tus mob typhoid
20,2,tus mob enteric
21,2,mob typhoid
22,2,paratyphoid
23,2,mob paratyphoid
24,3,endocarditis


In [16]:
def split_input(input_string):
    data = re.sub('\?', ' ?', input_string)
    return data.split(' ')

In [59]:
input_ = 'Cov kab mob ntawm tus mob typhoid fever zoo li cas?'

In [60]:
data = split_input(input_)

In [61]:
classification = np.argmax(q_type_classifier.predict(doc2vec_embeddings.infer_vector(data).reshape(1,32)))

In [62]:
def get_symptom_data(dsynID):
    sql_cmd = """SELECT baseName FROM relationDsynHasSosy
    JOIN sosy ON relationDsynHasSosy.sosyID=sosy.id
    WHERE dsynID={id_};""".format(id_=dsynID)
    cursor.execute(sql_cmd)
    for item in cursor:
        print(item[0])

In [63]:
if classification == 1:
    answer = None
    data_lc = [w.lower() for w in data]
    for item in dsynNames:
        if item[1].lower() in input_.lower():
            answer = item[0]
            break
    if answer == None:
        print("Tus mob no mas nrhiav tsis tau.")
    else:
        print("Tus mob no muaj cov tsos mob hauv qab no:")
        get_symptom_data(answer)
else:
    print("Tsis muaj cov information txog tus mob no.")

Tus mob no muaj cov tsos mob hauv qab no:
ua npaws
kub ib ce
mob tob hau
tsis qab los noj mov
ntuav
raws plab
tus po kheev o tuaj
tej thaj liab doog
cov thaj liab doog
tsob ntshav
mob hnyuv tws
mob chob nkuav
